In [1]:
import numpy as np
from numpy import asarray
import os
import glob
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, log_loss
from tqdm import tqdm

In [2]:
def initialiation(dimensions):

    parametres = {}
    C = len(dimensions)

    for c in range(1, C):
        parametres['W' + str(c)] = np.random.randn(dimensions[c], dimensions[c - 1] )
        parametres['b' + str(c)] = np.random.randn(dimensions[c], 1)
        
    return parametres

In [4]:
def forward_propagation(X, parametres):
    activations = {'A0' : X}

    C = len(parametres) // 2

    for c in range(1, C+1) :

        Z = parametres['W' + str(c)].dot(activations['A' + str(c-1)]) + parametres['b' + str(c)]
        activations['A' + str(c)] = 1 / (1 + np.exp(-Z))

    
    return activations

In [5]:
def back_propagation(y, activations, parametres):
    gradients = {}
    m = y.shape[1]
    C = len(parametres) // 2

    dZ = activations['A' + str(C)] - y

    for c in reversed(range(1, C+1)):
        gradients['dW' + str(c)] = 1/m * np.dot(dZ,activations['A' + str(c-1)].T)
        gradients['db' + str(c)] = 1/m * np.sum(dZ, axis=1, keepdims=True)
        if c > 1 :
            dZ = np.dot(parametres['W' + str(c)].T, dZ) * activations['A' + str(c-1)] * (1 - activations['A' + str(c-1)])

    return gradients

In [6]:
def update(gradients, parametres, learning_rate):
    C = len(parametres) // 2

    for c in range(1, C+1):
        parametres['W' + str(c)] = parametres['W' + str(c)] - learning_rate * gradients['dW' + str(c)]
        parametres['b' + str(c)] = parametres['b' + str(c)] - learning_rate * gradients['db' + str(c)]

    return parametres

In [7]:
def predict(X, parametres):
    activations = forward_propagation(X, parametres)
    C = len(parametres) // 2
    Af = activations['A' + str(C)]
    return Af >= 0.5

### creation du réseau de neuronnes

In [8]:
def deep_neural_network(X, y, hidden_layers = (32,32,32), learning_rate= 0.1, n_iter=1000):

    #initialisation W, b
    dimensions = list(hidden_layers)
    dimensions.insert(0, X.shape[0])
    dimensions.append(y.shape[0])
    np.random.seed(1)
    parametres = initialiation(dimensions)

     # tableau numpy contenant les futures accuracy et log_loss
    training_history = np.zeros((int(n_iter), 2))

    C = len(parametres) // 2
    
    #boucle d'apprentissage
    for i in tqdm(range(n_iter)):

        #activations
        activations = forward_propagation(X, parametres)
        gradients = back_propagation(y, activations, parametres)
        parametres = update(gradients, parametres, learning_rate)
        Af = activations['A' + str(C)]

        # calcul du log_loss et de l'accuracy
        training_history[i, 0] = (log_loss(y.flatten(), Af.flatten()))
        y_pred = predict(X, parametres)
        training_history[i, 1] = (accuracy_score(y.flatten(), y_pred.flatten()))    
        

    # Plot courbe d'apprentissage
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(training_history[:, 0], label='train loss')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(training_history[:, 1], label='train acc')
    plt.legend()
    plt.show()

    #return parametres


## DATASET

In [9]:
def open_image(path):
    image = None
    try:
        image = Image.open(path)
    except:
        print('Could not open file ' + path)
    return image

In [10]:
def image_to_array(image):
    return asarray(image)

In [11]:
def array_to_image(array):
    Image.fromarray(array)

In [12]:
def crop_image(image, new_size={'x': 1000, 'y': 1000}):
    width, height = image.size
    left, upper, right, lower = 0, 0, width, height

    if width > height:
        offset = int((width - height) / 2)
        left = offset
        right = offset + height
    else:
        offset = int((height - width) / 2)
        left = offset
        right = offset + width
    return image.crop((left, upper, right, lower)).resize((new_size['x'], new_size['y']))

In [13]:
def array_infos_match(arr, debug=False, size={'x': 1000, 'y': 1000}, img_type='uint8'):
    if debug:
        if arr is not None:
            print('Shape: %s' % (arr.shape,))
            print('Ratio: %s' % str(arr.shape[1] / arr.shape[0]))
            print('Type : %s' % arr.dtype)
            return True
        else:
            print('no image!')
            return False
    return arr is not None and arr.shape == (size['x'], size['y']) and arr.dtype == img_type

In [14]:
def show_image(image):
    image.show()

In [15]:
def show_array(array):
    pyplot.imshow(array)
    pyplot.show()

In [16]:
def load_images(directory, label, limit=-1, size={'x': 1000, 'y': 1000}, extension='.jpeg'):
    pattern = '' if label == 'healthy' else label #virus
    images = [] if not os.path.isdir(directory) else glob.glob(directory + '/*' + pattern + '*' + extension)
    res = []
    labels = []

    print('loading ' + label + ' pictures from ' + directory + ' ...')

    if 0 <= limit < len(images):
        images = images[:limit]

    for image in images:
        array = image_to_array(crop_image(open_image(image), size))
        #print(array.shape)
        if not array_infos_match(array, False, size):
            print('wrong size for: ' + image)
            continue
        res.append(array.reshape(-1))
        if label == 'healthy' :
            labels.append(0)
        else :
            if label == 'virus' :
                labels.append(1)
            else :
                labels.append(1)

    return {
        'images': res,
        'labels': labels
    }

## Chargement des images 

In [17]:
dataset_file = './assets/lungs_dataset.json'
assets_path = './assets/chest_Xray'
assets = None

if os.path.exists(assets_path):
    print('Loading assets from images')
    image_size = {'x': 100, 'y': 100}

    assets = {
        'test': {
            'healthy': load_images(assets_path + '/test/NORMAL', 'healthy', (-1), image_size),
            'bacteria': load_images(assets_path + '/test/PNEUMONIA', 'virus', (-1), image_size),
            'virus': load_images(assets_path + '/test/PNEUMONIA', 'bacteria', (-1), image_size)
        },
        'train': {
            'healthy': load_images(assets_path + '/train/NORMAL', 'healthy', 100, image_size),
            'bacteria': load_images(assets_path + '/train/PNEUMONIA', 'virus', 100, image_size),
            'virus': load_images(assets_path + '/train/PNEUMONIA', 'bacteria', 100, image_size)
        },
        #'val': {
        #    'healthy': load_images(assets_path + '/val/NORMAL', 'healthy'),
        #    'bacteria': load_images(assets_path + '/val/PNEUMONIA', 'virus'),
        #    'virus': load_images(assets_path + '/val/PNEUMONIA', 'bacteria')
        #}
    }

    print('Loading assets from images: done!')
else:
    print('assets not found !')

Loading assets from images
loading healthy pictures from ./assets/chest_Xray/test/NORMAL ...
loading virus pictures from ./assets/chest_Xray/test/PNEUMONIA ...
loading bacteria pictures from ./assets/chest_Xray/test/PNEUMONIA ...
loading healthy pictures from ./assets/chest_Xray/train/NORMAL ...
loading virus pictures from ./assets/chest_Xray/train/PNEUMONIA ...
wrong size for: ./assets/chest_Xray/train/PNEUMONIA\person1010_virus_1695.jpeg
wrong size for: ./assets/chest_Xray/train/PNEUMONIA\person1052_virus_1751.jpeg
wrong size for: ./assets/chest_Xray/train/PNEUMONIA\person1099_virus_1819.jpeg
wrong size for: ./assets/chest_Xray/train/PNEUMONIA\person1104_virus_1826.jpeg
wrong size for: ./assets/chest_Xray/train/PNEUMONIA\person1108_virus_1833.jpeg
loading bacteria pictures from ./assets/chest_Xray/train/PNEUMONIA ...
wrong size for: ./assets/chest_Xray/train/PNEUMONIA\person1035_bacteria_2969.jpeg
wrong size for: ./assets/chest_Xray/train/PNEUMONIA\person1073_bacteria_3008.jpeg
Loadi

## creation de matrice pour le modele

In [18]:
X_train = assets['train']['healthy']['images'] + assets['train']['bacteria']['images'] + assets['train']['virus']['images']
Y_train = assets['train']['healthy']['images'] + assets['train']['bacteria']['labels'] + assets['train']['virus']['labels']

X_train = np.array(X_train)
y_train = np.array(Y_train)

X = X_train.T
y = y_train.reshape((1,y_train.shape[0]))
y



C:\Users\sf690\AppData\Local\Temp\ipykernel_7376\3201816663.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y_train = np.array(Y_train)


array([[array([83, 90, 93, ..., 76, 80, 82], dtype=uint8),
        array([ 89,  92, 102, ..., 149, 161, 141], dtype=uint8),
        array([ 82,  83,  83, ..., 133, 133, 138], dtype=uint8),
        array([107, 113, 118, ..., 157, 147, 137], dtype=uint8),
        array([ 27,  21,  14, ..., 153, 150, 144], dtype=uint8),
        array([133, 134, 135, ..., 198, 199, 198], dtype=uint8),
        array([ 39,  37,  37, ..., 143, 144, 138], dtype=uint8),
        array([ 90, 129, 118, ..., 130, 120, 100], dtype=uint8),
        array([  0,   2,  44, ..., 158, 155, 134], dtype=uint8),
        array([158, 147, 127, ..., 147, 134, 129], dtype=uint8),
        array([68, 72, 70, ...,  0,  0,  0], dtype=uint8),
        array([ 68,  67,  61, ..., 119, 116, 109], dtype=uint8),
        array([76, 68, 61, ...,  0,  0,  0], dtype=uint8),
        array([108, 104, 123, ..., 142, 149, 132], dtype=uint8),
        array([31, 32, 31, ...,  0,  0,  0], dtype=uint8),
        array([ 50,  45,  38, ..., 186, 186, 187]

## phase d'entrainement

In [19]:
#deep_neural_network(X, y, hidden_layers = (32, 32, 32), learning_rate = 0.1, n_iter = 2000)

  0%|          | 0/2000 [00:00<?, ?it/s]C:\Users\sf690\AppData\Local\Temp\ipykernel_7376\2090253647.py:9: RuntimeWarning: overflow encountered in exp
  activations['A' + str(c)] = 1 / (1 + np.exp(-Z))


## phase test

In [ ]:
#deep_neural_network(X_test, y_test, hidden_layers = (32, 32, 32), learning_rate = 0.1, n_iter = 2000)